In [3]:
import os
import shutil
import random
from pathlib import Path

# Source directories
main_cat = "./data/PetImages/Cat"
main_dog = "./data/PetImages/Dog"
main_men = "./data/gender_dataset/men"
main_women = "./data/gender_dataset/women"

# Target directories
target_train_cat = "./data/train/cat"
target_train_dog = "./data/train/dog"
target_train_person = "./data/train/person"

target_test_cat = "./data/test/cat"
target_test_dog = "./data/test/dog"
target_test_person = "./data/test/person"

# Function to split and copy images
def split_and_copy(src_dirs, train_dir, test_dir, split_ratio=0.8):
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    all_files = []
    for src in src_dirs:
        all_files += [str(Path(src) / f) for f in os.listdir(src) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    random.shuffle(all_files)
    split_point = int(len(all_files) * split_ratio)
    train_files = all_files[:split_point]
    test_files = all_files[split_point:]

    print(f"Processing {len(all_files)} images → {len(train_files)} train / {len(test_files)} test")

    for files, target in [(train_files, train_dir), (test_files, test_dir)]:
        for f in files:
            try:
                shutil.copy(f, target)
            except Exception as e:
                print(f"Error copying {f}: {e}")

# Split cats
split_and_copy([main_cat], target_train_cat, target_test_cat)

# Split dogs
split_and_copy([main_dog], target_train_dog, target_test_dog)

# Split people (men + women together)
split_and_copy([main_men, main_women], target_train_person, target_test_person)


Processing 12499 images → 9999 train / 2500 test
Processing 12499 images → 9999 train / 2500 test
Processing 1668 images → 1334 train / 334 test


In [5]:
image_dir="data/train"
for label,class_dir in enumerate(os.listdir(image_dir)):
    print(label,class_dir)

0 person
1 cat
2 dog


In [8]:
import os
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

In [ ]:
class ImageDataset(Dataset):
    def __init__(self,image_dir,transform=None):
        self.image_dir=image_dir
        self.image_paths=[]
        self.labels=[]  ### labes 0/1/2
        self.class_name={}
        self.transform=transform

        for label,class_dir in enumerate(os.listdir(image_dir)):
            self.class_name[label]=class_dir
            class_path=os.path.join(image_dir,class_dir)
            for image_name in os.listdir(class_path):
                self.image_paths.append(os.path.join(class_path,image_name))
                self.labels.append(label)
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self,idx):
        img_path=self.image_paths[idx]
        image=Image.open(img_path).convert("RGB")
        label=self.labels[idx]
        if self.transform is not None:
            image=self.transfrom(image)
        return image,label

In [26]:
transform=transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor()
])

In [27]:
train_image_dir="/home/sachchida/gitrepo/computervision/cnn/data/train"
test_image_dir="/home/sachchida/gitrepo/computervision/cnn/data/test"

train_image_dataset=ImageDataset(image_dir=train_image_dir,transform=transform)
train_image_dataset=ImageDataset(image_dir=test_image_dir,transform=transform)

In [28]:
train_image_loader=DataLoader(dataset=train_image_dataset,batch_size=32,shuffle=True)
test_image_loader=DataLoader(dataset=train_image_dataset,batch_size=32,shuffle=True)

In [29]:
for images,labels in train_image_loader:
    print(images.shape, labels.shape)

AttributeError: 'ImageDataset' object has no attribute 'transfrom'